In [80]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import keras
import cv2

from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Convolution2D, Flatten, MaxPooling2D, Dropout
from keras.utils import np_utils

In [9]:
ds = pd.read_csv('data/mnist_train.csv',header=None)
data = ds.values
print data.shape
X_data = data[:, 1:]
y_data = data[:, 0]
print X_data.shape, y_data.shape

(60000, 785)
(60000, 784) (60000,)


In [38]:
dt = pd.read_csv('data/mnist_test.csv',header=None)
dv = dt.values
print dv.shape
X_test = dv[:, 1:]
y_test = dv[:, 0]
print X_test.shape, y_test.shape

(10000, 785)
(10000, 784) (10000,)


In [24]:

# X_data = X_data/255.0

X_data = (X_data-X_data.mean(axis=0))/(X_data.std(axis=0) + 1e-8)
y = np_utils.to_categorical(y_data)
# print y[4]
print X_data.shape, y.shape

(60000, 784) (60000, 10)


In [39]:
X_test = (X_test-X_test.mean(axis=0))/(X_test.std(axis=0) + 1e-8)
y_test = np_utils.to_categorical(y_test)
print X_test.shape, y_test.shape

(10000, 784) (10000, 10)


In [40]:
# split = int(0.85*X_data.shape[0])

# X_train = X_data[:split]
# y_train = y[:split]
# X_test = X_data[split:]
# y_test = y[split:]

# print X_train.shape, y_train.shape
# print X_test.shape, y_test.shape

<h1> Feed Forward Model using Keras </h1>

In [45]:
model = Sequential()

model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_7 (Dense)                  (None, 512)           401920      dense_input_4[0][0]              
____________________________________________________________________________________________________
activation_7 (Activation)        (None, 512)           0           dense_7[0][0]                    
____________________________________________________________________________________________________
dense_8 (Dense)                  (None, 10)            5130        activation_7[0][0]               
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 10)            0           dense_8[0][0]                    
Total params: 407050
______________________________________________________________________

In [46]:
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [47]:
print model.layers[0]

In [48]:
hist = model.fit(X_data, y,
         nb_epoch=30,
         shuffle=True,
         batch_size=128,
         validation_data=(X_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/30
60000/60000 [==============================] - 5s - loss: 0.3170 - acc: 0.9117 - val_loss: 0.1660 - val_acc: 0.9528
Epoch 2/30
60000/60000 [==============================] - 6s - loss: 0.1314 - acc: 0.9634 - val_loss: 0.1275 - val_acc: 0.9648
Epoch 3/30
60000/60000 [==============================] - 6s - loss: 0.0894 - acc: 0.9751 - val_loss: 0.1103 - val_acc: 0.9693
Epoch 4/30
60000/60000 [==============================] - 6s - loss: 0.0655 - acc: 0.9821 - val_loss: 0.0998 - val_acc: 0.9743
Epoch 5/30
60000/60000 [==============================] - 6s - loss: 0.0501 - acc: 0.9873 - val_loss: 0.0928 - val_acc: 0.9755
Epoch 6/30
60000/60000 [==============================] - 6s - loss: 0.0388 - acc: 0.9907 - val_loss: 0.0914 - val_acc: 0.9764
Epoch 7/30
60000/60000 [==============================] - 6s - loss: 0.0306 - acc: 0.9935 - val_loss: 0.0883 - val_acc: 0.9775
Epoch 8/30
60000/60000 [==============================] - 6s 

In [49]:
from keras.models import load_model
import h5py

model.save('tmodels/digit2.h5')  # creates a HDF5 file 'my_model.h5'
f = h5py.File('tmodels/digit2.h5', 'r+')
del f['optimizer_weights']
f.close()

In [54]:
for i in range(1,7):
    im = cv2.imread('timages/Digit'+str(i)+'.jpg',0)
    im = im.flatten()
    im = im[np.newaxis]
    print model.predict_classes(im,verbose=0)[0]

3
1
2
6
5
4


In [69]:
ds = pd.read_csv('data/mnist_train.csv',header=None)
data = ds.values
print data.shape
X_train = data[:, 1:]
y_train = data[:, 0]
X_train = X_train.reshape(60000,1,28,28)
print X_train.shape, y_train.shape

(60000, 785)
(60000, 1, 28, 28) (60000,)


In [70]:
dt = pd.read_csv('data/mnist_test.csv',header=None)
dv = dt.values
print dv.shape
X_test = dv[:, 1:]
y_test = dv[:, 0]
X_test = X_test.reshape(10000,1,28,28)
print X_test.shape, y_test.shape

(10000, 785)
(10000, 1, 28, 28) (10000,)


In [71]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [72]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [95]:
filters = 32
kernel_size = (3,3)
batch_size = 128
nb_epoch = 15

In [103]:
conv_model = Sequential()

conv_model.add(Convolution2D(filters, kernel_size[0], kernel_size[1], input_shape=(1, 28, 28)))
conv_model.add(Activation('relu'))
conv_model.add(Convolution2D(filters, kernel_size[0], kernel_size[1]))
conv_model.add(Activation('relu'))
conv_model.add(MaxPooling2D(pool_size=(2,2)))
conv_model.add(Dropout(0.25))
conv_model.add(Flatten())
conv_model.add(Dense(128))
conv_model.add(Activation('relu'))
conv_model.add(Dropout(0.5))
conv_model.add(Dense(10))
conv_model.add(Activation('softmax'))

conv_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_17 (Convolution2D) (None, 32, 26, 26)    320         convolution2d_input_14[0][0]     
____________________________________________________________________________________________________
activation_36 (Activation)       (None, 32, 26, 26)    0           convolution2d_17[0][0]           
____________________________________________________________________________________________________
convolution2d_18 (Convolution2D) (None, 32, 24, 24)    9248        activation_36[0][0]              
____________________________________________________________________________________________________
activation_37 (Activation)       (None, 32, 24, 24)    0           convolution2d_18[0][0]           
___________________________________________________________________________________________

In [96]:
conv_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [97]:
conv_model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 53s - loss: 0.0448 - acc: 0.9865 - val_loss: 0.0296 - val_acc: 0.9897
Epoch 2/15
60000/60000 [==============================] - 58s - loss: 0.0430 - acc: 0.9871 - val_loss: 0.0289 - val_acc: 0.9905
Epoch 3/15
60000/60000 [==============================] - 53s - loss: 0.0420 - acc: 0.9872 - val_loss: 0.0297 - val_acc: 0.9903
Epoch 4/15
60000/60000 [==============================] - 52s - loss: 0.0412 - acc: 0.9874 - val_loss: 0.0289 - val_acc: 0.9905
Epoch 5/15
60000/60000 [==============================] - 54s - loss: 0.0407 - acc: 0.9879 - val_loss: 0.0279 - val_acc: 0.9907
Epoch 6/15
60000/60000 [==============================] - 53s - loss: 0.0414 - acc: 0.9873 - val_loss: 0.0283 - val_acc: 0.9897
Epoch 7/15
60000/60000 [==============================] - 52s - loss: 0.0386 - acc: 0.9879 - val_loss: 0.0281 - val_acc: 0.9910
Epoch 8/15
60000/60000 [==============================

In [101]:
conv_model.save('tmodels/cDigit1.h5')  # creates a HDF5 file 'my_model.h5'
f = h5py.File('tmodels/cDigit1.h5', 'r+')
del f['optimizer_weights']
f.close()